<a href="https://colab.research.google.com/github/BenJMcCarty/processing_arrivals_excel/blob/master/Ben's_Hotel_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arrivals Excel Code Filter

- Sheraton Delaney Valley
- 03/24/20
- James M. Irving, Ph.D.
    - [Email Me](mailto:james.irving.phd@gmail.com)
    - [Meet with Me](https://go.oncehub.com/JamesIrvingExtendedHours)


## README


> ***This notebook is intended to load in an arrivals report from ____ (ask Ben) and return a filtered list containing only room numbers who contain the following service codes attached to their reservation.***
    
- **How to use this notebook.:**
    0. Upload files to analyze to google drive.
    1. Mount google drive and log into google drive.
    2. Change the `FOLDER` and `EXCEL_FILE` parmaters in the first cell below to match your current files.
        - Tip: Use the sidebar File viewer (Folder icon on left of screen) to browse files. (Note: Google Drive will be located in `/gdrive/My Drive/`
    
    3. **On Menu bar, click `Runtime` > `Run all`**
    4. Scroll down to ["You May Edit Below This Header"](#YOU-MAY-EDIT-BELOW-THIS-HEADER)


## EDIT `FOLDER` and `EXCEL_FILE` BELOW

In [1]:
## EDIT THESE TO MATCH ACTUAL FOLDER/FILE PATHS
FOLDER = 'drive/My Drive/Hotel Data/'
EXCEL_FILE = 'AA - 5-7-19.xlsx'


################### DO NOT EDIT BELOW THIS LINE ###################
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import os
files = os.listdir(FOLDER)

if EXCEL_FILE in files == False:
    print(f'[!] ERROR: Excel file {EXCEL_FILE} not found in {FOLDER}')
else:
    print(f'[i] SUCCESS: Excel file {EXCEL_FILE} found in {FOLDER}')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
[i] SUCCESS: Excel file AA - 5-7-19.xlsx found in drive/My Drive/Hotel Data/


# DO NOT EDIT BELOW THIS HEADER
___

In [0]:
try:
    from fsds_100719.imports import *
except:
    !pip install -U fsds_100719
    from fsds_100719.imports import *

pd.set_option('display.max_rows',999)
dp.clear_output()

In [0]:
def read_excel_file(filename,remove_header=True,
                    sheet_name='Report', skiprows=4):
    """Reads in an arrivals xlsx file, fill in blank row room numbers, 
    remove unumbered rooms at the top of the file
    
    Args:
        filname(str): full folder path and filename of xlsx to load.
        remove_header (bool): Drop rooms with missing room #s at top of sheet(Default is True)
        sheet_name (str): Name of xlsx sheet to load
        skiprows (int): Number of header rows to exclude from file (Default is 4)

    Returns:
        DataFrame: pandas DataFrame with 
        """
    ## Read in excel file, 
    df = pd.read_excel(filename,sheet_name=sheet_name, skiprows=skiprows)

    ## Fill Null Values with Room # from Row Above
    df['Number Type']=df['Number Type'].ffill()

    
    if remove_header:
        ## Remove Remaining Null Values from beginnging rows
        df = df[~df['Number Type'].isna()]#.head()

    return df


def process_data(df_raw,only_code_columns=False,
                 renamer_kws={}):
    """Processes raw loaded df to check 2 columns for all 
    service codes and return a dataframe with all room.
    Renames the columns as in renamer_dict: 
    renamer = {'Number Type':'Room Number', 
            'Unnamed: 14': 'Service Codes 1',
            'Stat GT Guest Name':'Service Codes 2'} """
    df = df_raw.copy()

    ## Rename columns of interest.
    renamer = {'Number Type':'Room Number', 
               'Unnamed: 14': 'Service Codes 1',
               'Stat GT Guest Name':'Service Codes 2'}
    renamer.update(renamer_kws)
    df = df.rename(mapper=renamer,axis=1)

    ## Join together columns with service codes
    code_cols = ['Service Codes 1','Service Codes 2']
    df[code_cols] =  df[code_cols].fillna('').astype(str)
    df['All Codes'] = df['Service Codes 1']+' '+ df['Service Codes 2']

    ## Make Room Number into Integers
    tf = df['Room Number'].str.isnumeric()

    ## Cut off Footer Text Totals
    df = df.loc[tf!=False] 
    
    ## Covert Rooms to integers
    df['Room Number'] =df['Room Number'].astype(int)
   
   ## Control if all or just code columns are returned
    if only_code_columns:
        
        # Select only data of interest
        df = df[['Room Number','All Codes']]
    return df


def combine_room_rows(df_):
    """Joins the overflow row of service codes to the new "Combined Codes" column."""
    df = df_.copy()

    ## Get room number index
    rooms = df.groupby('Room Number').groups

    ## Save Series to list for pd.concat
    clean_df = []

    ## For each room, create one row with all combined codes
    for rm_num, idx in rooms.items():
        df_temp = df.loc[idx]

        ## If no overflow second row:
        if len(idx)==1:
            df_temp['Combined Codes'] = df_temp['All Codes'].copy()

        ## If second or more overflow row, combine codes
        elif len(idx)>1:
            df_temp['Combined Codes'] = ' '.join(df_temp.loc[idx,'All Codes'])

        ## Add codes to clean_df


        clean_df.append(df_temp)

    ## Concatenate final df and slice out columns
    final_df = pd.concat(clean_df)[['Room Number','Combined Codes']]

    ## Drop any duplicated rooms
    final_df.drop_duplicates(inplace=True)
    
    return final_df  


In [0]:
def filter_codes(DF,codes_to_discard = ['PARK','CPAR','NP'],
                 only_filtered_cols=True):
    """Filters out reservations containing codes and creates T/F columns for each tag
    """
    ## FILTER OUT RESERVATIONS WITH CODES

    ## Create TF Column for if combined codes contains it
    for code in codes_to_discard:
        DF[f'HAS {code}'] = DF['Combined Codes'].str.contains(code)

    ## Get T/F for combined code columns
    DF['DISCARD'] = DF[[col for col in DF.columns if "HAS" in col]].sum(axis=1) >0 

    ## Set room numbers as axes
    # DF.set_index('Room Number',inplace=True)
    DF.sort_values(by='Room Number',inplace=True)

    ## Keep only columns without tags 
    if only_filtered_cols:
        DF = DF[DF['DISCARD']==False]
    
    return DF.reset_index()

def complete_process(excel_file, folder = r'drive/My Drive/Hotel Data/',
                    codes_to_discard = ['PARK','CPAR','NP'],only_filtered_cols=True,
                    save_file=True,save_filename=None):
    """Performs full process using all of the functions above.
    
    Args:
        excel_file(str): excel file name with .xlsx and NO folderpath
        folder (str) : Folder containing excel file (Defaults to 'drive/My Drive/Data Sets/Hotel Data/')
    
    Returns:
        DataFrame (str): Pandas DataFrame with list of room numbers without service codes of interest
    """
    filename = folder+excel_file
    print(f"Loading {filename}")
    NEW_FILENAME = f"{folder}{excel_file.split('.')[-2]}{'_CODES.xlsx'}"

    raw_df = read_excel_file(filename)
    df = process_data(raw_df,False)
    df_final = combine_room_rows(df)
    df_final2 = filter_codes(df_final,codes_to_discard=codes_to_discard,
                             only_filtered_cols=only_filtered_cols)
    df_final2 = df_final2[['Room Number','DISCARD','HAS PARK', 'HAS CPAR', 'HAS NP', 'Combined Codes', ]]

    ## Replace bad text in codes
    replace_me = ['Schedule/Rate','Rate',':']
    for repl in replace_me:
        df_final2['Combined Codes']= df_final2['Combined Codes'].apply(lambda x: x.replace(repl,''))#.strip())
    if save_filename is None:
        df_final2.to_excel(NEW_FILENAME)

    return df_final2

# YOU MAY EDIT BELOW THIS HEADER

___

In [8]:
## By Default, the function will save the file and will only return the columns of interest
df1 = complete_process(EXCEL_FILE,save_file=False,folder=FOLDER)
df1.head()

Loading drive/My Drive/Hotel Data/AA - 5-7-19.xlsx


,Room Number,DISCARD,HAS PARK,HAS CPAR,HAS NP,Combined Codes
0,306,False,False,False,False,I2 N4 P9 TK
1,308,False,False,False,False,TD
2,314,False,False,False,False,P9 TK
3,315,False,False,False,False,TK
4,319,False,False,False,False,E1 L3 TK


In [9]:
df = complete_process(EXCEL_FILE,save_file=True,only_filtered_cols=False)
df.head() #.style.highlight_max(subset='DISCARD',color='green').hide_index()

Loading drive/My Drive/Hotel Data/AA - 5-7-19.xlsx


,Room Number,DISCARD,HAS PARK,HAS CPAR,HAS NP,Combined Codes
0,301,True,False,True,False,CPAR FB1 N3 TD R4 K6 N4 I2
1,305,True,True,False,False,D4 PARK TD
2,306,False,False,False,False,I2 N4 P9 TK
3,308,False,False,False,False,TD
4,311,True,True,False,False,K1 PARK TK
